In [1]:
!pip install SimpleITK
!pip install pyradiomics

     |████████████████████████████████| 48.4 MB 310 kB/s eta 0:00:01
  Using cached pyradiomics-3.0.1.tar.gz (34.5 MB)
     |████████████████████████████████| 6.9 MB 3.4 MB/s eta 0:00:01
  Using cached pykwalify-1.8.0-py2.py3-none-any.whl (24 kB)
  Using cached ruamel.yaml-0.17.21-py3-none-any.whl (109 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
     |████████████████████████████████| 539 kB 17.3 MB/s eta 0:00:01
  Created wheel for pyradiomics: filename=pyradiomics-3.0.1-cp39-cp39-linux_x86_64.whl size=117356 sha256=720b409177cd6939bc133d18d2cf9db15240eed3713f562dad73f96356a8144e
  Stored in directory: /home/calculo3/.cache/pip/wheels/62/0f/06/e8bdcc276fc814981e11a2fc4da7ad0da0a7e68a60fe90e747
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=8938248a265550816eda52320a2c5e5b7e5fdc52246ff12c4931f058e98582db
  Stored in directory: /home/calculo3/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built pyra

In [1]:
from __future__ import print_function

import logging
import os

import pandas 
import SimpleITK as sitk

import radiomics
from radiomics import featureextractor

In [4]:
#ESTE Es El ARCHIVO PARA EXTRAER FEATURES CON PYRADIOMICS


outPath = ''
paramsPath= ''
inputPath = ''
inputCSV = os.path.join(inputPath, 'test_images.csv')
outputFilepath = os.path.join(outPath, 'radiomicsFeatures.csv')
progress_filename = os.path.join(outPath, 'pyrad_log.txt')
params = os.path.join(paramsPath, 'Params_dxa.yaml')

 # Configure logging
rLogger = logging.getLogger('radiomics')

  # Set logging level
  # rLogger.setLevel(logging.INFO)  # Not needed, default log level of logger is INFO

  # Create handler for writing to log file
handler = logging.FileHandler(filename=progress_filename, mode='w')
handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
rLogger.addHandler(handler)

  # Initialize logging for batch log messages
logger = rLogger.getChild('batch')

  # Set verbosity level for output to stderr (default level = WARNING)
radiomics.setVerbosity(logging.INFO)

logger.info('pyradiomics version: %s', radiomics.__version__)
logger.info('Loading CSV')
    #flists = pandas.read_csv(inputCSV).T
try:
    # Use pandas to read and transpose ('.T') the input data
    # The transposition is needed so that each column represents one test case. This is easier for iteration over
    # the input cases
    flists = pandas.read_csv(inputCSV).T
except Exception:
    logger.error('CSV READ FAILED', exc_info=True)
    exit(-1)

logger.info('Loading Done')
logger.info('Patients: %d', len(flists.columns))
    
extractor = featureextractor.RadiomicsFeatureExtractor(params)
'''if os.path.isfile(params):
        extractor = featureextractor.RadiomicsFeatureExtractor(params)
    else:  # Parameter file not found, use hardcoded settings instead
        settings = {}
        settings['binWidth'] = 25
        settings['resampledPixelSpacing'] = None  # [3,3,3]
        settings['interpolator'] = sitk.sitkBSpline
        settings['enableCExtensions'] = True
        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
        # extractor.enableInputImages(wavelet= {'level': 2})
'''
logger.info('Enabled input images types: %s', extractor.enabledImagetypes)
logger.info('Enabled features: %s', extractor.enabledFeatures)
logger.info('Current settings: %s', extractor.settings)

  # Instantiate a pandas data frame to hold the results of all patients
results = pandas.DataFrame()

for entry in flists:  # Loop over all columns (i.e. the test cases)
    logger.info("(%d/%d) Processing Patient (Image: %s, Mask: %s)",
        entry + 1,
        len(flists),
        flists[entry]['Image'],
        flists[entry]['Mask'])
       
    imageFilepath = flists[entry]['Image']
    maskFilepath = flists[entry]['Mask']
        #label = flists[entry].get('Label', None)

    label =255

    if (imageFilepath is not None) and (maskFilepath is not None):
        featureVector = flists[entry]  # This is a pandas Series
        featureVector['Image'] = os.path.basename(imageFilepath)
        featureVector['Mask'] = os.path.basename(maskFilepath)

        try:
        # PyRadiomics returns the result as an ordered dictionary, which can be easily converted to a pandas Series
        # The keys in the dictionary will be used as the index (labels for the rows), with the values of the features
        # as the values in the rows.
            print(imageFilepath)
            print(maskFilepath)
            result = pandas.Series(extractor.execute(imageFilepath, maskFilepath, label=255))
            featureVector = featureVector.append(result)
        except Exception:
            logger.error('FEATURE EXTRACTION FAILED:', exc_info=True)

      # To add the calculated features for this case to our data frame, the series must have a name (which will be the
      # name of the column.
        print(entry)
        featureVector.name = entry
      # By specifying an 'outer' join, all calculated features are added to the data frame, including those not
      # calculated for previous cases. This also ensures we don't end up with an empty frame, as for the first patient
      # it is 'joined' with the empty data frame.
        results = results.join(featureVector, how='outer')  # If feature extraction failed, results will be all NaN


logger.info('Extraction complete, writing CSV')
  # .T transposes the data frame, so that each line will represent one patient, with the extracted features as columns
results.T.to_csv(outputFilepath, index=False, na_rep='NaN')
logger.info('CSV writing complete')
    
    #print(results.T['original_glszm_LargeAreaLowGrayLevelEmphasis'])

pyradiomics version: v3.0.1
Loading CSV
Loading Done
Patients: 4
Loading parameter file Params_dxa.yaml
Enabled input images types: {'Original': {}}
Enabled features: {'firstorder': [], 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Current settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True}
(1/4) Processing Patient (Image: test_images/L1.tif, Mask: test_images/L1_mask.tif)
Calculating features with label: 255
Loading image and mask
Adding image type "Original" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Computing glrlm
Comp

test_images/L1.tif
test_images/L1_mask.tif
0
test_images/L2.tif
test_images/L2_mask.tif


Adding image type "Original" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
/tmp/ipykernel_127825/3208723563.py:88: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  featureVector = featureVector.append(result)
(3/4) Processing Patient (Image: test_images/L3.tif, Mask: test_images/L3_mask.tif)
Calculating features with label: 255
Loading image and mask
Adding image type "Original" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
/tmp/ipykernel_127825/3208723563.py:88: FutureWarning: The se

1
test_images/L3.tif
test_images/L3_mask.tif
2
test_images/L4.tif
test_images/L4_mask.tif
3


CSV writing complete


In [2]:
pandas.set_option("display.max_rows", None, "display.max_columns", None)
print(results.T['original_ngtdm_Complexity'])

NameError: name 'results' is not defined